### AJAX
##### AJAX也就是XHR技術的另一個稱號
##### 練習抓取kkday的網站資料
##### kkday有使用AJAX的技術，因此直接抓網址抓不到資料，要找出真正抓得到資料的網址
##### 如果要判斷網站是否有用AJAX，可以看網頁的html有沒有包含頁面的資訊，沒有的話就很可能是用AJAX的技術
##### 可以用F12開啟網站的開發者工具，查看網站的資料、找出需要的資訊，這次的在network-->XHR，透過preview來找出包含列表資訊的地方，然後在headers的general可以看到真正包含資料的request url

In [3]:
import urllib.request as req
import bs4
import requests
import json
from urllib.parse import quote
import pandas as pd
import json
from pathlib import Path
base_path = Path(r'%notebook').absolute().parent.parent.parent
with open(fr"{base_path}\config.json", "r") as f:
    config = json.load(f)

In [4]:
# KKday網站指定城市的一日遊票券資訊
class Kkday:
 
    #建構式
    def __init__(self, city_name, page_headers, cat):
        self.city_name = quote(city_name, encoding='utf-8')  # 城市名稱屬性，要將我們要查的城市轉換成URL編碼
        self.page_headers = page_headers # 這個API要多一點headers資訊
        self.cat = cat  # 類別屬性，看是要一日遊還是2日遊
        self.page_url = f"https://www.kkday.com/zh-tw/product/ajax_productlist?keyword={self.city_name}&cat={self.cat}"
        self.total_data = pd.DataFrame()
    
    # 用來取得總共有多少頁面要抓
    def get_target_page_amount(self):
        response = requests.get(self.page_url, headers=self.page_headers)
        self.total_page = response.json()["total_page"]

    # 抓每個頁面的資料            
    def scrape(self, page_number):
	
        result = []  # 回傳結果
		
        if self.city_name:  # 如果城市名稱非空值
 
            # 取得傳入城市的所有一日遊票券
            url = self.page_url + f"&page={page_number}"
            response = requests.get(url, headers=self.page_headers)
				
            # 資料
            activities = response.json()["data"]
                
            for activity in activities:
    
               # 票券名稱
                title = activity["name"]
 
                # 票券詳細內容連結
                link = activity["url"]
 
                # 票券價格
                price = activity["price"]
 
                # 最早可使用日期
                booking_date = activity["earliest_sale_date"]
 
                # 評價
                star = activity["rating_star"]
                    
                result.append(
                        dict(title=title, link=link, price=price, booking_date=booking_date, star=star, source="KKday"))
 
        return result
    
    def get_all_page_data(self):
        # 用來取得總共有多少頁面要抓
        self.get_target_page_amount()
        # for迴圈遞迴抓每個頁面的資料
        for i in range(1,self.total_page+1):
            result = self.scrape(i)
            result = pd.DataFrame(result)
            result['page'] = i
            self.total_data = pd.concat([self.total_data, result])

            

		


In [5]:
page_headers={
    'User-Agent':config["request_headers"]["User-Agent"],
    'Accept':config["request_headers"]["Accept"],
    'Accept-Encoding':config["request_headers"]["Accept-Encoding"],
    'Accept-Language':config["request_headers"]["Accept-Language"],
    'Sec-Ch-Ua-Platform':config["request_headers"]["Sec-Ch-Ua-Platform"],
    'Sec-Fetch-Dest':config["request_headers"]["Sec-Fetch-Dest"],
    }
cat = 'TAG_4_4'

demo = Kkday("新竹市", page_headers, cat)
demo.get_all_page_data()
result_df = demo.total_data
result_df

,title,link,price,booking_date,star,source,page
0,【獨家套票:84折】新竹綠世界生態農場門票|贈綠世界糖朝指定飲料折價券,https://www.kkday.com/zh-tw/product/101863,340,20231220,4.68,KKday,1
1,台中&新竹|薰衣草森林|門票,https://www.kkday.com/zh-tw/product/101859,188,20231217,4.70,KKday,1
2,【2024 跨年日出推薦】新竹雪霸農場迎曙光&雲霧步道一日遊|台北&桃園&新竹出發,https://www.kkday.com/zh-tw/product/155616,1780,20240101,0.00,KKday,1
3,【特價促銷】新竹司馬庫斯一日來回接送|台北/新北/桃園/新竹出發,https://www.kkday.com/zh-tw/product/138228,1680,20231222,4.81,KKday,1
4,苗栗 | 中階健行 | 加里山登山一日遊,https://www.kkday.com/zh-tw/product/104094,2180,20231230,4.77,KKday,1
...,...,...,...,...,...,...,...
8,新竹一日遊|觀霧森林遊樂區&雪霸休閒農場,https://www.kkday.com/zh-tw/product/19760,1899,20231217,3.29,KKday,6
9,【秋日限定】新竹海柿山盟農遊體驗一日|台北/台中出發,https://www.kkday.com/zh-tw/product/154110,1800,20231217,0.00,KKday,6
0,新竹人氣健行景點|霞喀羅古道一日縱走|台北市區出發,https://www.kkday.com/zh-tw/product/100407,2700,20231217,3.88,KKday,7
1,台灣包車新竹一日遊 | 綠世界生態農場&北埔老街&合興車站,https://www.kkday.com/zh-tw/product/114413,5000,20231217,4.82,KKday,7
